# Notebook to correct radiometry in Planet imagery using SciKit histogram matching

Rainey Aberle

Summer 2022

In [ ]:
import os
import glob
import numpy as np
import rasterio as rio
import matplotlib.pyplot as plt
from skimage.exposure import match_histograms

# path to Planet images
im_path = '/Users/raineyaberle/Research/PhD/Wolverine/imagery/Planet/2021-04-20_2021-08-25/PSScene4Band/'
out_path = im_path+'../im_histograms/'

In [ ]:
# -----Grab image file names
ims = os.chdir(im_path) # change directory
im_names = glob.glob('*SR_clip.tif') # load all .tif file names
im_names.sort() # sort file names by date
im_names

In [ ]:
# -----load 'correct' image
# load image
im1_name = im_names[1]
im1 = rio.open(im1_name)
# read image bands
im_scalar = 10000
b1 = im1.read(1) / im_scalar
g1 = im1.read(2) / im_scalar
r1 = im1.read(3) / im_scalar
nir1 = im1.read(4) / im_scalar
# define grid
im1_x = np.linspace(im1.bounds.left, im1.bounds.right, num=np.shape(b1)[1])
im1_y = np.linspace(im1.bounds.top, im1.bounds.bottom, num=np.shape(b1)[0])

# -----load 'incorrect' image
im2_name = im_names[3]
im2 = rio.open(im2_name)
# read image bands
b2 = im2.read(1) / im_scalar
g2 = im2.read(2) / im_scalar
r2 = im2.read(3) / im_scalar
nir2 = im2.read(4) / im_scalar
# define grid
im2_x = np.linspace(im2.bounds.left, im2.bounds.right, num=np.shape(b2)[1])
im2_y = np.linspace(im2.bounds.top, im2.bounds.bottom, num=np.shape(b2)[0])

# -----apply histogram matching to each band
b2_matched = match_histograms(b2, b1)
g2_matched = match_histograms(g2, g1)
r2_matched = match_histograms(r2, r1)
nir2_matched = match_histograms(nir2, nir1)

In [ ]:
# -----plot RGB images and histograms
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(16,10), gridspec_kw={'height_ratios': [3,1]})
plt.rcParams.update({'font.size': 12, 'font.serif': 'Arial'})
# incorrect image
ax1.imshow(np.dstack([r2, g2, b2]),
            extent=(np.min(im2_x)/1000, np.max(im2_x/1000), np.min(im2_y)/1000, np.max(im2_y)/1000))
ax1.set_xlabel('Easting [km]')
ax1.set_ylabel('Northing [km]')
ax1.set_title('Incorrect image')
h1_nir = ax4.hist(nir2.flatten(), color='purple', bins=100, alpha=0.5, label='NIR')
h1_b = ax4.hist(b2.flatten(), color='blue', bins=100, alpha=0.5, label='blue')
h1_g = ax4.hist(g2.flatten(), color='green', bins=100, alpha=0.5, label='green')
h1_r = ax4.hist(r2.flatten(), color='red', bins=100, alpha=0.5, label='red')
ax4.set_xlabel('Surface reflectance')
ax4.set_ylabel('Pixel counts')
ax4.set_ylim(0,np.max([h1_nir[0][1:], h1_g[0][1:], h1_r[0][1:], h1_b[0][1:]])+5000)
ax4.grid()
ax3.legend(loc='right')
# reference image
ax2.imshow(np.dstack([r1, g1, b1]),
            extent=(np.min(im1_x)/1000, np.max(im1_x/1000), np.min(im1_y)/1000, np.max(im1_y)/1000))
ax2.set_xlabel('Easting [km]')
ax2.set_title('Reference image')
h2_nir = ax5.hist(nir1.flatten(), color='purple', bins=100, alpha=0.5, label='NIR')
h2_b = ax5.hist(b1.flatten(), color='blue', bins=100, alpha=0.5, label='blue')
h2_g = ax5.hist(g1.flatten(), color='green', bins=100, alpha=0.5, label='green')
h2_r = ax5.hist(r1.flatten(), color='red', bins=100, alpha=0.5, label='red')
ax5.set_xlabel('Surface reflectance')
ax5.set_ylim(0,np.max([h2_nir[0][1:], h2_g[0][1:], h2_r[0][1:], h2_b[0][1:]])+5000)
ax5.grid()
# histogram-matched image
ax3.imshow(np.dstack([r2_matched, g2_matched, b2_matched]),
            extent=(np.min(im2_x)/1000, np.max(im2_x/1000), np.min(im2_y)/1000, np.max(im2_y)/1000))
ax3.set_xlabel('Easting [km]')
ax3.set_title('Histogram-matched image')
h3_nir = ax6.hist(nir2_matched.flatten(), color='purple', bins=100, alpha=0.5, label='NIR')
h3_b = ax6.hist(b2_matched.flatten(), color='blue', bins=100, alpha=0.5, label='blue')
h3_g = ax6.hist(g2_matched.flatten(), color='green', bins=100, alpha=0.5, label='green')
h3_r = ax6.hist(r2_matched.flatten(), color='red', bins=100, alpha=0.5, label='red')
ax6.set_xlabel('Surface reflectance')
ax6.set_ylim(0,np.max([h3_nir[0][1:], h3_g[0][1:], h3_r[0][1:], h3_b[0][1:]])+5000)
ax6.grid()
fig.tight_layout()
plt.show()